<a href="https://colab.research.google.com/github/bforoura/Transformers/blob/main/nlp_transformers_ch8_ex1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title A Positional Encoding Example

# Gensim is a free open-source Python library for representing documents as semantic vectors
!pip install gensim==3.8.3


# An open-source machine learning library, a scientific computing framework, and a 
# script language based on the Lua programming language.
import torch

# A suite of libraries and programs for statistical language processing
import nltk

# Punktis is a Sentence Tokenizer that divides a text into a list of sentences by using 
# an unsupervised algorithm to build a model for abbreviation words, collocations, and 
# words that start sentences.
nltk.download('punkt')



In [2]:
#@title Necessary imports

import math
import numpy as np
from nltk.tokenize import sent_tokenize, word_tokenize 
import gensim 
from gensim.models import Word2Vec 
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import warnings 
warnings.filterwarnings(action = 'ignore') 



In [3]:
#@title Upload the data file and parse it

dprint=0 # prints outputs if set to 1, default=0

#‘text.txt’ will reside locally in your Google colab file system 
!curl -L https://raw.githubusercontent.com/PacktPublishing/Transformers-for-Natural-Language-Processing/main/Chapter01/text.txt --output "test.txt"

sample = open("test.txt", "r") 
s = sample.read() 

# processing escape characters 
f = s.replace("\n", " ") 

data = [] 

# sentence parsing 
for i in sent_tokenize(f): 
	temp = [] 
	# tokenize the sentence into words 
	for j in word_tokenize(i): 
		temp.append(j.lower()) 
	data.append(temp) 
 



  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 24151  100 24151    0     0   259k      0 --:--:-- --:--:-- --:--:--  259k


In [6]:
#@title Create the Skip Gram model

model = gensim.models.Word2Vec(data, min_count = 1, size = 512, window = 5, sg = 1)

# 1-The 2-black 3-cat 4-sat 5-on 6-the 7-couch 8-and 9-the 10-brown 11-dog 12-slept 13-on 14-the 15-rug.
word1='black'
pos1=2

word2='brown'
pos2=10

a=model[word1]
b=model[word2]

if(dprint==1):
        print(a)


In [7]:
#@title Compute cosine similarity

dot = np.dot(a, b)

norma = np.linalg.norm(a)
normb = np.linalg.norm(b)

cos = dot / (norma * normb)

aa = a.reshape(1,512) 
ba = b.reshape(1,512)
cos_lib = cosine_similarity(aa, ba)
